# Keras

Keras — открытая нейросетевая библиотека, написанная на языке Python. Она представляет собой надстройку над фреймворками Deeplearning4j, TensorFlow и Theano.[1][2] Нацелена на оперативную работу с сетями глубинного обучения, при этом спроектирована так, чтобы быть компактной, модульной и расширяемой. Она была создана как часть исследовательских усилий проекта ONEIROS (англ. Open-ended Neuro-Electronic Intelligent Robot Operating System),[3] а ее основным автором и поддерживающим является Франсуа Шолле (фр. François Chollet), инженер Google.

Планировалось что Google будет поддерживать Keras в основной библиотеке TensorFlow, однако Шолле выделил Keras в отдельную надстройку, так как согласно концепции Keras является скорее интерфейсом, чем сквозной системой машинного обучения. Keras предоставляет высокоуровневый, более интуитивный набор абстракций, который делает простым формирования нейронных сетей не зависимо от используемой на нижнем уровне библиотеки научных вычислений.[4] Microsoft работает над добавлением к Keras и нижнеуровневых библиотек CNTK[en].[5]

https://ru.wikipedia.org/wiki/Keras

In [1]:
from keras.layers import Dense
from keras.models import Sequential
import keras
import os

/home/borsh/develop/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## MNIST 
Загрузим MNIST

In [2]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

X, y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

Тестовые и тренировачные данные

In [3]:
X_train, y_train,X_test,  y_test = X[:60000], y[:60000], X[60000:], y[60000:]
X_train = X_train / 255.
X_test = X_test / 255.


img_rows, img_cols = 28, 28

X_train2D = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test2D = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)


кодируем метки (label) в K-мерный массив, необходимый шаг для тренировки, когда результатом является метка. Для данных MNIST метка - число от 0 до 9 - цифра, изображенная на изображении

In [4]:
num_classes = 10
# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.to_categorical(y_test, num_classes)


## Dense neural network DNN
Строим нейронную сеть

In [5]:
model = Sequential()

model.add(Dense(units=300, activation='relu', input_dim=784))

model.add(Dense(units=100, activation='relu', input_dim=300))
model.add(Dense(units=32, activation='relu', input_dim=100))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Тренируем

In [50]:
model.fit(X_train, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
60000/60000 [==============================] - 12s 192us/step - loss: 0.3416 - acc: 0.9042
Epoch 2/10
60000/60000 [==============================] - 13s 223us/step - loss: 0.2936 - acc: 0.9294
Epoch 3/10
60000/60000 [==============================] - 11s 182us/step - loss: 0.3061 - acc: 0.9357
Epoch 4/10
60000/60000 [==============================] - 11s 181us/step - loss: 0.3089 - acc: 0.9412
Epoch 5/10
60000/60000 [==============================] - 11s 189us/step - loss: 0.3240 - acc: 0.9415
Epoch 6/10
60000/60000 [==============================] - 13s 210us/step - loss: 0.3233 - acc: 0.9404
Epoch 7/10
60000/60000 [==============================] - 11s 186us/step - loss: 0.3203 - acc: 0.9420
Epoch 8/10
60000/60000 [==============================] - 11s 184us/step - loss: 0.3080 - acc: 0.9437
Epoch 9/10
60000/60000 [==============================] - 11s 187us/step - loss: 0.3170 - acc: 0.9440
Epoch 10/10
60000/60000 [==============================] - 13s 220us/step - loss: 

Если долго ждать, можно загрузить из файла/

In [6]:
from keras.models import load_model
model = load_model(os.path.expanduser('~/model/keras_mnist_dnn.h5'))

Проверяем

In [7]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=128)
loss_and_metrics

10000/10000 [==============================] - 0s 37us/step


[0.43501708616993856, 0.9484]

Сохраняем

In [8]:
model.save(os.path.expanduser('~/model/keras_mnist_dnn.h5'))

## Convolutional neural network CNN


Строим сеть


In [9]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12
input_shape = (img_rows, img_cols, 1)


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



 и тренируем её

In [6]:
model.fit(X_train2D, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          )

NameError: name 'batch_size' is not defined

Если долго ждать, можно загрузить уже натренированную

In [7]:
from keras.models import load_model
model = load_model(os.path.expanduser('~/model/keras_mnist_cnn.h5'))

Проверяем

In [8]:
loss_and_metrics = model.evaluate(X_test2D, Y_test, batch_size=128)
loss_and_metrics[1]

10000/10000 [==============================] - 7s 748us/step


0.9904

Сохраняем

In [9]:
model.save(os.path.expanduser('~/model/keras_mnist_cnn.h5'))

## Cifar10

In [2]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 10

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [4]:

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [ ]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 222s 4ms/step - loss: 1.8477 - acc: 0.3250 - val_loss: 1.5564 - val_acc: 0.4421
Epoch 2/100
50000/50000 [==============================] - 215s 4ms/step - loss: 1.5178 - acc: 0.4503 - val_loss: 1.3653 - val_acc: 0.5206
Epoch 3/100
50000/50000 [==============================] - 210s 4ms/step - loss: 1.3755 - acc: 0.5046 - val_loss: 1.2502 - val_acc: 0.5531
Epoch 4/100
13184/50000 [======>.......................] - ETA: 2:25 - loss: 1.2811 - acc: 0.5470

In [ ]:
\;'pmodel.save(os.path.expanduser('~/model/keras_cifar10.h5'))


# Задание

Создать нейронную сеть, которая бы позволяла предсказать стоимость дома на основе датасета california_housing

In [25]:
from sklearn.datasets import california_housing
data = california_housing.fetch_california_housing()
X, y = data['data'], data['target']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((15480, 8), (5160, 8), (15480,), (5160,))

Подсказка:
Эта задача на регрессию. Для того, чтобы определить нейронную сеть, которая бы решала эту проблему, последний слой должен быть с линейной активаторной функцией. 

    model.add(Dense(1, activation='linear'))